# IdP-proxyの削除

---

構築したIdP-proxyを削除します。

## パラメータの指定

### VCCアクセストークンの入力

VCノードを削除するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
ERROR - config vc failed: http_status(403)
ERROR - 2021/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

### グループ名

削除対象となるVCPのUnitGroup名を指定します。

対象となり得るUnitGroupを確認するために group_varsファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/

 UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'IdPproxy'

ugroup_name = 

## 構築環境の削除

起動したVCノードを削除します。

現在のUnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

現在のVCノードの状態を確認します。

In [ ]:
ugroup = vcp.get_ugroup(ugroup_name)
ugroup.df_nodes()

UnitGroupに属する全てのVCノードとUnitGroupの削除を行います。

In [ ]:
ugroup.cleanup()

削除後の UnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

## Ansible設定のクリア

削除した環境に対応するAnsibleの設定をクリアします。

### group_varsファイル

group_varsファイルをリネームします。

In [ ]:
!mv group_vars/{ugroup_name} group_vars/.{ugroup_name}.bak

### インベントリ

インベントリから削除対象のグループを除去します。

In [ ]:
 %run scripts/group.py
remove_group_from_inventory_yml(ugroup_name)

# 削除前との差分を表示します。
! ! diff -u inventory.yml.bak inventory.yml